In [1]:
import pandas as pd
import nltk
veri = pd.read_csv("onislenmis_veriler.csv") #Gerekli importlari yapip onislenmis verimi df degiskenine aktardim.

In [2]:
df = pd.DataFrame(veri)
df["cumleler"] #Verileri hatirlamak amaciyla baktim.
tf = df["cumleler"].apply(lambda x : pd.value_counts(x.split(" "))).sum(axis=0).reset_index()
tf.columns = ["words","tf"]
top_20_kelime =tf.sort_values("tf", ascending=False).astype(str).head(20) #En cok gecen 20 kelime sayilariyla beraber ayiklanip yazdirildi.

In [3]:
yeni_top_20 = top_20_kelime["words"].tolist()

In [4]:
yeni_top_20


['kisi',
 'yok',
 'adam',
 'buyuk',
 'erdogan',
 'cumhurbaskani',
 'tayyip',
 'turkiye',
 'son',
 'tek',
 'basbakan',
 'oy',
 'iyi',
 'gun',
 'zaman',
 'bugun',
 'ilk',
 'recep',
 'ulkenin',
 'secim']

In [5]:
#pip install gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
word2vec_model = Word2Vec(sentences=[sentence.split() for sentence in df['cumleler']], vector_size=100, window=5, min_count=1, workers=4)

In [7]:
df['sentence_vectors'] = df['cumleler'].apply(lambda sentence: [word2vec_model.wv[word] for word in sentence.split()])


In [8]:
df['similarities'] = df['sentence_vectors'].apply(lambda vectors: cosine_similarity(vectors))

In [9]:
df[['cumleler', 'similarities']] #Benzerlik analizi

,cumleler,similarities
0,bilmem belli donem secilmis ust duzey devlet m...,"[[1.0, 0.9530684, 0.94028735, 0.8845941, 0.941..."
1,cumhurbaskanligi yeminin anlamini bilmeyen gec...,"[[1.0, 0.32954547, 0.44048083, 0.89860845, 0.9..."
2,yeni reklam yuzu,"[[1.0, 0.82171446, 0.77576846], [0.82171446, 0..."
3,baskanlik sistemini ulkeye kisi,"[[1.0000001, 0.93418837, 0.9829614, 0.99208385..."
4,dedem tvde gordugunde cikti dolu demiyor,"[[0.99999994, 0.29318893, 0.23966292, 0.302052..."
...,...,...
13937,akp iktidarinda yabancilara toprak,"[[1.0, 0.80151653, 0.37598145, 0.68018246], [0..."
13938,istanbul topraklarini yabancilara satmak,"[[0.99999994, 0.22566557, 0.4098936, 0.5974073..."
13939,yabancilara satisi,"[[1.0000001, -0.003744225], [-0.003744225, 1.0]]"
13940,yabancilarin bol bol toprak satin almasi,"[[0.9999999, 0.38795546, 0.38795543, 0.306655,..."


In [10]:
from nltk.tokenize import word_tokenize #model egitme

In [11]:
tokenized_data = [word_tokenize(sentence) for sentence in df["cumleler"]]

In [12]:
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

In [13]:
model.save("egitilmis_model.bin")

In [14]:
model = Word2Vec.load("egitilmis_model.bin") 

In [15]:
similar_words_dict = {}

In [16]:
for word in yeni_top_20: #egitilen modele gore benzer kelimeleri bulma
    similar_words = model.wv.most_similar(word, topn=5)
    similar_words_dict[word] = [similar_word for similar_word, _ in similar_words]

In [17]:
for word, similar_words in similar_words_dict.items():
    print(f"{word}: {similar_words}")


kisi: ['bugun', 'oy', 'yok', 'son', 'iyi']
yok: ['zaman', 'adam', 'son', 'gun', 'kisi']
adam: ['son', 'yok', 'iyi', 'kisi', 'ulkenin']
buyuk: ['zaman', 'yok', 'kisi', 'oy', 'son']
erdogan: ['tayyip', 'erdogani', 'erdoganin', 'erdogana', 'adi']
cumhurbaskani: ['buyuk', 'basbakan', 'yok', 'son', 'ulke']
tayyip: ['erdogan', 'recep', 'erdogani', 'erdoganin', 'erdogana']
turkiye: ['buyuk', 'turk', 'ilk', 'bugun', 'zaman']
son: ['adam', 'yok', 'kisi', 'ulke', 'tek']
tek: ['kisi', 'son', 'ulkenin', 'adam', 'yok']
basbakan: ['iyi', 'ayni', 'cumhurbaskani', 'sadece', 'yok']
oy: ['kisi', 'buyuk', 'adam', 'yok', 'son']
iyi: ['zaman', 'kisi', 'yok', 'adam', 'son']
gun: ['yok', 'kisi', 'zaman', 'sadece', 'buyuk']
zaman: ['yok', 'iyi', 'buyuk', 'kisi', 'son']
bugun: ['kisi', 'yok', 'zaman', 'adam', 'gun']
ilk: ['yok', 'kisi', 'bugun', 'turk', 'son']
recep: ['erdogan', 'tayyip', 'erdogani', 'bey', 'erdogandir']
ulkenin: ['yok', 'tek', 'adam', 'iyi', 'son']
secim: ['son', 'sadece', 'zaman', 'ulke', 'i

In [22]:
"""from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Önceden belirlenmiş cümleler
secilen_cumleler = [
    "cumhurbaskanligi yeminin anlamini bilmeyen gecen kisilik kendinde olmadigini pesin pesin kabul kisidir",
    "milyonlarca insan aciklama beklerken sarayina ejder meyveli yudumluyor sanirim",
    "hayatta gun cumhurbaskanligi adayligi basbakan koltugundan istifa bilmek istedigim kisidir onsuz yasamak guzel bence",
    "yurtdisinda yasayan buyuk hizmetler goturen reisimsi disari yurda kalmiyor turkiye ulke olsaydi habire turkiyede tesekkur gonlum vermiyor edilmeyecek laflar ediyorum turkiye gormedi reisimsi",
    "turkiye cumhuriyeti cumhurbaskani rahat rahat atip tutuyor insani olduren milyonlarca insani evinden edip kaciran katil ruh hastasi esadin sucu yok taraftan amerikanin rusyanin avrupanin sucu yok turkiye teror karsi karsi savasiyor gelen geldigini bilen kisi katil ruh hastasi mezhepci esad sirf sunni insanlari ordusuyla olduruyor"
]

# 'cumleler' sütununu seçme
cumleler = veri['cumleler'].tolist()

# TfidfVectorizer kullanarak metinleri vektörlere dönüştürme
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(cumleler + secilen_cumleler)

# Cümleler arasındaki benzerliği ölçme
benzerlik_matrisi = cosine_similarity(vectors)

for i, ornek_cümle in enumerate(secilen_cumleler):
    benzerlik_skorları = benzerlik_matrisi[len(cumleler) + i][:len(cumleler)]
    
    # Kendisiyle eşleşen benzerlik skorlarını filtreleme
    benzerlik_skorları = [(index, skor) for index, skor in enumerate(benzerlik_skorları) if cumleler[index] != ornek_cümle]
    
    # Benzerlik skorlarını sıralama ve en yüksek üç benzerlik skorunu al
    en_benzer_cümleler_index = sorted(benzerlik_skorları, key=lambda x: x[1], reverse=True)[:3]
    
    # Sonuçları yazdırma
    print(f"\n{str(i+1)}. Örnek Cümle: {ornek_cümle}")
    print("En Benzer Üç Cümle:")
    for index, skor in en_benzer_cümleler_index:
        print(f"- {cumleler[index]} (Benzerlik Skoru: {round(skor, 2)})")"""


1. Örnek Cümle: cumhurbaskanligi yeminin anlamini bilmeyen gecen kisilik kendinde olmadigini pesin pesin kabul kisidir
En Benzer Üç Cümle:
- pesin (Benzerlik Skoru: 0.61)
- cumhurbaskanligi (Benzerlik Skoru: 0.22)
- gecen (Benzerlik Skoru: 0.22)

2. Örnek Cümle: milyonlarca insan aciklama beklerken sarayina ejder meyveli yudumluyor sanirim
En Benzer Üç Cümle:
- nicedir ortalarda yoktur ejder meyveli muhtemelen (Benzerlik Skoru: 0.34)
- sarayinda gelen yudumluyor olabilir (Benzerlik Skoru: 0.23)
- insan (Benzerlik Skoru: 0.22)

3. Örnek Cümle: hayatta gun cumhurbaskanligi adayligi basbakan koltugundan istifa bilmek istedigim kisidir onsuz yasamak guzel bence
En Benzer Üç Cümle:
- guzel gun (Benzerlik Skoru: 0.28)
- tek istedigim insan huzurlu yasamak (Benzerlik Skoru: 0.27)
- cumhurbaskanligi secimi sonrasinda gormek istedigim basbakan (Benzerlik Skoru: 0.27)

4. Örnek Cümle: yurtdisinda yasayan buyuk hizmetler goturen reisimsi disari yurda kalmiyor turkiye ulke olsaydi habire turkiyed

In [23]:
from textblob import TextBlob
veri_seti = pd.read_csv("onislenmis_veriler.csv")

# Metin sütununu seçme
metinler = veri_seti['cumleler'].tolist()

# Duygu analizi yapma ve sonuçları saklama
duygu_analizleri = []

for metin in metinler:
    blob = TextBlob(metin)
    duygu_durumu = blob.sentiment.polarity
    
    if duygu_durumu > 0:
        sonuc = "Olumlu"
    elif duygu_durumu < 0:
        sonuc = "Olumsuz"
    else:
        sonuc = "Nötr"
    
    duygu_analizi = {"Metin": metin, "Duygu Durumu": duygu_durumu, "Sonuç": sonuc}
    duygu_analizleri.append(duygu_analizi)

# Duygu analizi sonuçlarını DataFrame'e çevirme
duygu_analiz_df = pd.DataFrame(duygu_analizleri)

# Sonuçları yazdırma
print(duygu_analiz_df)

                                                   Metin  Duygu Durumu Sonuç
0      bilmem belli donem secilmis ust duzey devlet m...           0.0  Nötr
1      cumhurbaskanligi yeminin anlamini bilmeyen gec...           0.0  Nötr
2                                       yeni reklam yuzu           0.0  Nötr
3                        baskanlik sistemini ulkeye kisi           0.0  Nötr
4               dedem tvde gordugunde cikti dolu demiyor           0.0  Nötr
...                                                  ...           ...   ...
13937                 akp iktidarinda yabancilara toprak           0.0  Nötr
13938           istanbul topraklarini yabancilara satmak           0.0  Nötr
13939                                 yabancilara satisi           0.0  Nötr
13940           yabancilarin bol bol toprak satin almasi           0.0  Nötr
13941                                    babalar satariz           0.0  Nötr

[13942 rows x 3 columns]


In [28]:
duygu_analiz_df.value_counts()

Metin                                                                                                                                             Duygu Durumu  Sonuç
gorsel                                                                                                                                            0.0           Nötr     45
recep tayyip erdogan                                                                                                                              0.0           Nötr     19
imla                                                                                                                                              0.0           Nötr     18
kaynak                                                                                                                                            0.0           Nötr     18
yap                                                                                                                                               

In [29]:
olumlu_sayisi = duygu_analiz_df[duygu_analiz_df['Sonuç'] == 'Olumlu'].shape[0]
olumsuz_sayisi = duygu_analiz_df[duygu_analiz_df['Sonuç'] == 'Olumsuz'].shape[0]
notr_sayisi = duygu_analiz_df[duygu_analiz_df['Sonuç'] == 'Nötr'].shape[0]

# Sonuçları yazdırma
print(f"Olumlu Sayısı: {olumlu_sayisi}")
print(f"Olumsuz Sayısı: {olumsuz_sayisi}")
print(f"Nötr Sayısı: {notr_sayisi}")

Olumlu Sayısı: 78
Olumsuz Sayısı: 27
Nötr Sayısı: 13837
